# Import library

In [ ]:
!pip install --upgrade pip
!pip install optree
!pip install anytree

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
import os
import cv2
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

warnings.filterwarnings("ignore", category=DeprecationWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
np.random.seed(54)
tf.random.set_seed(54)
train_dir = '/kaggle/input/Original Images/Original Images/'
checkpoint_filepath = '/kaggle/working/best_self_built_model.keras'

## List of used pretrained model

In [ ]:
from keras.applications import VGG16
from keras.applications import ResNet50
from keras.applications import DenseNet121
from keras.applications import InceptionV3
from keras.applications import MobileNetV2
from keras.applications import EfficientNetB0

# Data preparation

In [ ]:
image_size = 128

In [ ]:
# Using ImageDataGenerator for data augmentation
generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1 
)

train_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="training"  # This is for training data
)

val_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="validation"  # This is for validation data
)

# Evaluate the model on the test set
test_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="validation"  # Use a portion of the data for testing
)

In [ ]:
# Get the list of classes
classes = list(train_ds.class_indices.keys())
print(classes)

In [ ]:
def plot_roc_curves(model, test_ds, num_classes=31, batch_size=32, num_samples=1000):
    y_true = []
    y_pred = []

    for i, (images, labels) in enumerate(test_ds):
        if len(y_true) >= num_samples:
            break
        y_true.extend(np.argmax(labels, axis=1))
        y_pred.extend(model.predict(images, batch_size=batch_size, verbose=0))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Binarize the true labels
    y_true_bin = label_binarize(y_true, classes=[i for i in range(num_classes)])

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Plot ROC curves
    plt.figure(figsize=(10, 8))
    plt.plot(fpr["micro"], tpr["micro"], color='darkorange', lw=2,
             label='Micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]))

    # Plot ROC curve for each class
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], lw=2, alpha=0.3,
                 label='Class {0} (AUC = {1:0.2f})'.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right", bbox_to_anchor=(1.5, 0))
    plt.show()

# CNN model

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to avoid memory allocation issues
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs configured.")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs found. Training will use the CPU.")

## Build model

In [ ]:
IMAGE_SIZE = [image_size, image_size, 3]
NUM_CLASSES = 31

model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=IMAGE_SIZE))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# Print model summary
model.summary()

## Set up checkpoint for fetching best model

In [ ]:
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='train_loss',
    save_best_only=True,
    mode='min',
    verbose=0
)

## Train

In [ ]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    batch_size=32,
    callbacks=[checkpoint_callback]
)

# Load the best model after training
best_model = tf.keras.models.load_model(checkpoint_filepath)

## Result

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
train_loss, train_accuracy = model.evaluate(train_ds)
print(f"Training Accuracy: {train_accuracy*100: .2f}")
print(f"Training Loss: {train_loss: .2f}")

validation_loss, validation_accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {validation_accuracy*100: .2f}")
print(f"Validation Loss: {validation_loss: .2f}")

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy*100: .2f}")

In [ ]:
plot_roc_curves(model, test_ds)

# Fine-tune pretrained model

In [ ]:
image_size = 128

## Dataset preparation

In [ ]:
# Using ImageDataGenerator for data augmentation
generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1 
)

train_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="training"  # This is for training data
)

val_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="validation"  # This is for validation data
)

# Evaluate the model on the test set
test_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="validation"  # Use a portion of the data for testing
)

## Build model

In [ ]:
IMAGE_SIZE = [image_size, image_size, 3]

# Loading the weights of pretrained model without the top layer. These weights are trained on Imagenet dataset.
# pretrained_model = VGG16(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = ResNet50(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
pretrained_model = DenseNet121(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = InceptionV3(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = MobileNetV2(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = EfficientNetB0(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)


# This will exclude the initial layers from the training phase as they have already been trained.
for layer in pretrained_model.layers:
    layer.trainable = False

x = Flatten()(pretrained_model.output)
x = Dense(128, activation='relu')(x)  # We can add a new fully connected layer but it will increase the execution time.
# x = Dropout(0.15)(x)  # Add a Dropout layer with a dropout rate of 0.3
x = Dense(31, activation='softmax')(x)  # Adding the output layer with softmax function as this is a multi-class classification problem.

model = Model(inputs=pretrained_model.input, outputs=x)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print("Model created and compiled.")

# Summary of the model
model.summary()

## Set up checkpoint for fetching best model

In [ ]:
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='train_loss',
    save_best_only=True,
    mode='min',
    verbose=0
)

## Train

In [ ]:
history = model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds,
    batch_size=32,
    callbacks=[checkpoint_callback]
)

best_model = tf.keras.models.load_model(checkpoint_filepath)

## Result

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
train_loss, train_accuracy = model.evaluate(train_ds)
print(f"Training Accuracy: {train_accuracy*100: .2f}")
print(f"Training Loss: {train_loss: .2f}")

validation_loss, validation_accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {validation_accuracy*100: .2f}")
print(f"Validation Loss: {validation_loss: .2f}")

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy*100: .2f}")

In [ ]:
plot_roc_curves(model, test_ds)

## Save model

In [ ]:
best_model.save('best_fine_tune_model.h5')
from IPython.display import FileLink

# Create a download link
FileLink('FRM.h5')

# Cropping image

In [ ]:
# Get the directory containing Haar cascade files
cascade_dir = cv2.data.haarcascades

# Path to the Haar cascade file for frontal face detection
cascade_file = os.path.join(cascade_dir, 'haarcascade_frontalface_default.xml')

# Check if the cascade file exists
if os.path.isfile(cascade_file):
    print("Haar cascade file found:", cascade_file)
else:
    print("Haar cascade file not found. Downloading...")
    cv2_base_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/"
    cascade_url = cv2_base_url + 'haarcascade_frontalface_default.xml'
    os.system(f"wget {cascade_url} -P {cascade_dir}")
    print("Haar cascade file downloaded successfully.")

# Now, you can use cascade_file as the filter_path in your code.
filter_path = cascade_file

In [ ]:
# Function to detect faces and crop them from an image
def detect_and_crop_faces(image):
    face_cascade = cv2.CascadeClassifier(filter_path)
    faces = face_cascade.detectMultiScale(image, 1.3, 5)
    cropped_faces = []
    for (x, y, w, h) in faces:
        cropped_faces.append(image[y:y+h, x:x+w])
    return cropped_faces

In [ ]:
# Path to the dataset directory
dataset_dir = "/kaggle/input/Original Images/Original Images"
# Path to store the cropped images
cropped_dataset_dir = "/kaggle/working/CroppedImages"
# Path to store the split train and test sets
train_dir = os.path.join(cropped_dataset_dir, "train")
test_dir = os.path.join(cropped_dataset_dir, "test")

# Create directories for train and test sets
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Common size to which all face images will be resized
common_size = (128, 128)

# Iterate through each subdirectory (each person's folder)
for subdir in os.listdir(dataset_dir):
    subdir_path = os.path.join(dataset_dir, subdir)
    if os.path.isdir(subdir_path):
        # Create corresponding subdirectories in train and test folders
        train_subdir_path = os.path.join(train_dir, subdir)
        test_subdir_path = os.path.join(test_dir, subdir)
        os.makedirs(train_subdir_path, exist_ok=True)
        os.makedirs(test_subdir_path, exist_ok=True)

        # Get the list of image files in the subdirectory
        image_files = [f for f in os.listdir(subdir_path) if f.endswith('.jpg')]

        # Iterate through each image in the subdirectory
        for image_name in image_files:
            image_path = os.path.join(subdir_path, image_name)
            # Read the image
            img = cv2.imread(image_path)
            # Detect and crop faces from the image (function detect_and_crop_faces to be defined)
            faces = detect_and_crop_faces(img)
            # Resize each face to a common size before appending to the list
            for idx, face in enumerate(faces):
                if face is not None:
                    resized_face = cv2.resize(face, common_size)
                    # Decide whether to put the image in train or test set
                    if np.random.rand() < 0.9:  # 90% train, 10% test
                        save_path = os.path.join(train_subdir_path, f"{image_name}_{idx}.jpg")
                    else:
                        save_path = os.path.join(test_subdir_path, f"{image_name}_{idx}.jpg")
                    # Save the cropped face image
                    cv2.imwrite(save_path, resized_face)
print('dataset created')

In [ ]:
%matplotlib inline
cropped_img = mpimg.imread('/kaggle/working/CroppedImages/train/Billie Eilish/Billie Eilish_1.jpg_0.jpg')
original_img = mpimg.imread('/kaggle/input/Original Images/Original Images/Billie Eilish/Billie Eilish_1.jpg')

In [ ]:
imgplot = plt.imshow(original_img)

In [ ]:
imgplot = plt.imshow(cropped_img)

In [ ]:
import os
from anytree import Node, RenderTree

def create_directory_tree(root_path, parent=None):
    """
    Recursively create a directory tree structure using AnyTree.
    """
    node = Node(os.path.basename(root_path), parent=parent)
    if os.path.isdir(root_path):
        for item in sorted(os.listdir(root_path)):
            item_path = os.path.join(root_path, item)
            create_directory_tree(item_path, parent=node)

def print_directory_tree(root_path):
    """
    Print the directory tree structure using AnyTree.
    """
    root = Node(os.path.basename(root_path))
    create_directory_tree(root_path, root)

# Define the root directory
root_dir = "/kaggle/working/CroppedImages"

# Print the directory tree
print_directory_tree(root_dir)

## Dataset preparation

In [ ]:
image_size = 128

In [ ]:
train_dir = '/kaggle/working/CroppedImages/train'
test_dir = '/kaggle/working/CroppedImages/test'

# Using ImageDataGenerator for data augmentation
generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1  # 10% of the data will be used for validation
)

# Load and split the data into training and validation sets
train_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="training"  # This is for training data
)

val_ds = generator.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="validation"  # This is for validation data
)

# Get the list of classes
classes = list(train_ds.class_indices.keys())
print("Classes in training data:", classes)

# Load test data
test_ds = generator.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=32,
    subset="training"  # This is for test data
)

## Build model

In [ ]:
# Ensure TensorFlow is set to use GPU (if available)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
IMAGE_SIZE = [image_size, image_size, 3]  # we will keep the image size as (128, 128). You can increase the size for better results.

# loading the weights of pretrained model without the top layer. These weights are trained on Imagenet dataset.
# pretrained_model = VGG16(input_shape = IMAGE_SIZE, weights = 'imagenet', include_top = False)
# pretrained_model = ResNet50(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = DenseNet121(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = InceptionV3(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
# pretrained_model = MobileNetV2(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
pretrained_model = EfficientNetB0(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)

# this will exclude the initial layers from the training phase as they have already been trained.
for layer in pretrained_model.layers:
    layer.trainable = False

x = Flatten()(pretrained_model.output)
x = Dense(128, activation = 'relu')(x)  # we can add a new fully connected layer but it will increase the execution time.
x = Dense(31, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi-label classification problem.

model = Model(inputs = pretrained_model.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model created")

model.summary()

## Train model

In [ ]:
BATCH_SIZE = 32
history = model.fit(train_ds, epochs=30, validation_data=val_ds, batch_size=BATCH_SIZE)

## Result

In [ ]:
train_loss, train_accuracy = model.evaluate(train_ds)
print(f"Training Accuracy: {train_accuracy*100: .2f}")
print(f"Training Loss: {train_loss: .2f}")

validation_loss, validation_accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {validation_accuracy*100: .2f}")
print(f"Validation Loss: {validation_loss: .2f}")

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy*100: .2f}")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:
plot_roc_curves(model, test_ds)

In [ ]:
model.save('FRM_Crop.h5')

# Metric learning

## Metric learning CNN model

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
image_size_dim = 128

### Build dataset

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Preparation
data_dir = '/kaggle/input/Original Images/Original Images'
image_size = (image_size_dim, image_size_dim)

def load_images(data_dir, image_size):
    labels = []
    images = []
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        if os.path.isdir(label_path):
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                image = load_img(image_path, target_size=image_size, color_mode='grayscale')
                image = img_to_array(image) / 255.0
                images.append(image)
                labels.append(label_dir)
    return np.array(images), np.array(labels)

images, labels = load_images(data_dir, image_size)
label_to_int = {label: idx for idx, label in enumerate(np.unique(labels))}
int_labels = np.array([label_to_int[label] for label in labels])

# Generate pairs
def generate_pairs(images, labels):
    pairs = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]
        positive_idx = np.where(labels == label)[0]
        negative_idx = np.where(labels != label)[0]
        pos = images[np.random.choice(positive_idx)]
        neg = images[np.random.choice(negative_idx)]
        pairs += [[current_image, pos], [current_image, neg]]
        pair_labels += [1, 0]
    return np.array(pairs), np.array(pair_labels)

pairs, pair_labels = generate_pairs(images, int_labels)

# Split into training and test sets
(pairs_train, pairs_test, labels_train, labels_test) = train_test_split(pairs, pair_labels, test_size=0.2)

### Build Model

In [ ]:
# Model Architecture
def create_base_network(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (7, 7), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128))
    return model

input_shape = image_size + (1,)
base_network = create_base_network(input_shape)

input_a = tf.keras.Input(shape=input_shape)
input_b = tf.keras.Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = tf.keras.layers.Lambda(lambda embeddings: tf.keras.backend.sqrt(
    tf.keras.backend.sum(tf.keras.backend.square(embeddings[0] - embeddings[1]), axis=-1, keepdims=True))
)([processed_a, processed_b])

model = tf.keras.Model(inputs=[input_a, input_b], outputs=distance)

# Loss Function
def contrastive_loss(y_true, y_pred, margin=1):
    square_pred = tf.keras.backend.square(y_pred)
    margin_square = tf.keras.backend.square(tf.keras.backend.maximum(margin - y_pred, 0))
    return tf.keras.backend.mean(y_true * square_pred + (1 - y_true) * margin_square)

model.compile(loss=contrastive_loss, optimizer='adam')

# Training
history = model.fit([pairs_train[:, 0], pairs_train[:, 1]], labels_train, 
                    validation_data=([pairs_test[:, 0], pairs_test[:, 1]], labels_test),
                    epochs=30, batch_size=128)

### Evaluate Model

In [ ]:
# Evaluation
y_pred = model.predict([pairs_test[:, 0], pairs_test[:, 1]])
y_pred = np.where(y_pred < 0.5, 1, 0)
accuracy = accuracy_score(labels_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {history.history["loss"][-1]:.4f}')

# Plot training & validation loss values
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Predict distances on the test set
predicted_distances = model.predict([pairs_test[:, 0], pairs_test[:, 1]])

# Compute ROC curve and AUC
fpr, tpr, _ = roc_curve(labels_test, predicted_distances)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

print(f"Micro-average ROC curve area (AUC): {roc_auc:.2f}")

## Metric learning using pretrain model

### Dataset preparation

In [ ]:
image_size_dim = 128

In [ ]:
# from tensorflow.keras.applications.vgg16 import preprocess_input
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Data Preparation
data_dir = '/kaggle/input/Original Images/Original Images'
image_size = (image_size_dim, image_size_dim, 3)

def load_images(data_dir, image_size):
    labels = []
    images = []
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        if os.path.isdir(label_path):
            for image_name in os.listdir(label_path):
                image_path = os.path.join(label_path, image_name)
                image = load_img(image_path, target_size=image_size, color_mode='rgb')
                image = img_to_array(image)
                image = preprocess_input(image)
                images.append(image)
                labels.append(label_dir)
    return np.array(images), np.array(labels)

images, labels = load_images(data_dir, image_size)
label_encoder = LabelEncoder()
int_labels = label_encoder.fit_transform(labels)

# Generate pairs
def generate_pairs(images, labels):
    pairs = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]
        positive_idx = np.where(labels == label)[0]
        negative_idx = np.where(labels != label)[0]
        pos = images[np.random.choice(positive_idx)]
        neg = images[np.random.choice(negative_idx)]
        pairs += [[current_image, pos], [current_image, neg]]
        pair_labels += [1, 0]
    return np.array(pairs), np.array(pair_labels)

pairs, pair_labels = generate_pairs(images, int_labels)

# Split into training and test sets
(pairs_train, pairs_test, labels_train, labels_test) = train_test_split(pairs, pair_labels, test_size=0.2)

### Build Model

In [ ]:
# Model Architecture
def create_base_network(input_shape):
#     pretrained_model = VGG16(input_shape=input_shape, weights = 'imagenet', include_top = False)
#     pretrained_model = DenseNet121(input_shape=input_shape, weights='imagenet', include_top=False)
#     pretrained_model = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False)
    pretrained_model = MobileNetV2(input_shape=input_shape, weights='imagenet', include_top=False)
#     pretrained_model = EfficientNetB0(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
    x = layers.Flatten()(pretrained_model.output)
    x = layers.Dense(128)(x)
    model = models.Model(inputs=pretrained_model.input, outputs=x)
    return model

input_shape = image_size
base_network = create_base_network(input_shape)

input_a = tf.keras.Input(shape=input_shape)
input_b = tf.keras.Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = tf.keras.layers.Lambda(lambda embeddings: tf.keras.backend.sqrt(
    tf.keras.backend.sum(tf.keras.backend.square(embeddings[0] - embeddings[1]), axis=-1, keepdims=True))
)([processed_a, processed_b])

model = tf.keras.Model(inputs=[input_a, input_b], outputs=distance)

# Loss Function
def contrastive_loss(y_true, y_pred, margin=1):
    square_pred = tf.keras.backend.square(y_pred)
    margin_square = tf.keras.backend.square(tf.keras.backend.maximum(margin - y_pred, 0))
    return tf.keras.backend.mean(y_true * square_pred + (1 - y_true) * margin_square)

# Compile the model with accuracy metric
model.compile(loss=contrastive_loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

### Training

In [ ]:
history = model.fit([pairs_train[:, 0], pairs_train[:, 1]], labels_train, 
                    validation_data=([pairs_test[:, 0], pairs_test[:, 1]], labels_test),
                    epochs=50, batch_size=64)

### Result

In [ ]:
y_pred = model.predict([pairs_test[:, 0], pairs_test[:, 1]])
y_pred = np.where(y_pred < 0.5, 1, 0)
accuracy = accuracy_score(labels_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {history.history["loss"][-1]:.4f}')

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# Predict distances on the test set
predicted_distances = model.predict([pairs_test[:, 0], pairs_test[:, 1]])

# Compute ROC curve and AUC
fpr, tpr, _ = roc_curve(labels_test, predicted_distances)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

print(f"Micro-average ROC curve area (AUC): {roc_auc:.2f}")